In [1]:
import tensorflow as tf
import cv2
import numpy as np
from glob import glob
import os

In [2]:
sess = tf.Session()
sess.__enter__() # equivalent to `with sess:`
tf.saved_model.loader.load(sess, ["vgg16"], '../data/vgg')
    
#     writer = tf.summary.FileWriter("tflogs/")
#     writer.add_graph(sess.graph)
print(tf.global_variables())

INFO:tensorflow:Restoring parameters from b'../data/vgg/variables/variables'
[<tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>, <tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>, <tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>, <tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'conv3_2/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Variable 'conv3_2/biases:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'conv3_3/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Variable 'c

In [3]:
graph = tf.get_default_graph()
vgg_input_tensor_name = 'image_input:0'
vgg_keep_prob_tensor_name = 'keep_prob:0'
vgg_layer3_out_tensor_name = 'layer3_out:0'
vgg_layer4_out_tensor_name = 'layer4_out:0'
vgg_layer7_out_tensor_name = 'layer7_out:0'

image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

In [4]:
print(image_input)
print(keep_prob)
print(layer3_out)
print(layer4_out)
print(layer7_out)

Tensor("image_input:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("keep_prob:0", dtype=float32)
Tensor("layer3_out:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("layer4_out:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("layer7_out:0", shape=(?, ?, ?, 4096), dtype=float32)


In [5]:
# print all tensor names
tensors = [n.name for n in tf.get_default_graph().as_graph_def().node]
print(tensors)

['save/RestoreV2_29/shape_and_slices', 'save/RestoreV2_29/tensor_names', 'save/RestoreV2_28/shape_and_slices', 'save/RestoreV2_28/tensor_names', 'save/RestoreV2_27/shape_and_slices', 'save/RestoreV2_27/tensor_names', 'save/RestoreV2_26/shape_and_slices', 'save/RestoreV2_26/tensor_names', 'save/RestoreV2_25/shape_and_slices', 'save/RestoreV2_25/tensor_names', 'save/RestoreV2_24/shape_and_slices', 'save/RestoreV2_24/tensor_names', 'save/RestoreV2_23/shape_and_slices', 'save/RestoreV2_23/tensor_names', 'save/RestoreV2_22/shape_and_slices', 'save/RestoreV2_22/tensor_names', 'save/RestoreV2_21/shape_and_slices', 'save/RestoreV2_21/tensor_names', 'save/RestoreV2_20/shape_and_slices', 'save/RestoreV2_20/tensor_names', 'save/RestoreV2_19/shape_and_slices', 'save/RestoreV2_19/tensor_names', 'save/RestoreV2_18/shape_and_slices', 'save/RestoreV2_18/tensor_names', 'save/RestoreV2_17/shape_and_slices', 'save/RestoreV2_17/tensor_names', 'save/RestoreV2_16/shape_and_slices', 'save/RestoreV2_16/tensor

In [6]:
## try to get fc layers
fc6 = graph.get_tensor_by_name('fc6/Conv2D:0')
fc7 = graph.get_tensor_by_name('fc7/Conv2D:0')
pool5 = graph.get_tensor_by_name('pool5:0')
print(fc6)
print(fc7)
print(pool5)

Tensor("fc6/Conv2D:0", shape=(?, ?, ?, 4096), dtype=float32)
Tensor("fc7/Conv2D:0", shape=(?, ?, ?, 4096), dtype=float32)
Tensor("pool5:0", shape=(?, ?, ?, 512), dtype=float32)


### log metadata to get shape of tensors in graph

In [7]:
writer = tf.summary.FileWriter("tfgraphs/")
writer.add_graph(sess.graph)
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

# get an image
image_shape = (224, 224)
data_folder='../data/data_road/training'
image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
image_file = image_paths[0]
img = cv2.imread(image_file)
img = cv2.resize(img, image_shape[::-1])
img = np.array(img)
img = img[None,:] # add dimension for batches
print('img shape: {}'.format(img.shape))

# x, _ = get_batches_fn(1)
fc6,fc7,out = sess.run([fc6,fc7,layer7_out], feed_dict={image_input: img, keep_prob: 1},
                options=run_options,
                run_metadata=run_metadata)

writer.add_run_metadata(run_metadata, 'step0')
print('output tensor: {}'.format(out.shape))

img shape: (1, 224, 224, 3)
output tensor: (1, 7, 7, 4096)


In [8]:
print('pool5: {}'.format(.shape))
print('fc6: {}'.format(fc6.shape))
print('fc7: {}'.format(fc7.shape))

SyntaxError: invalid syntax (<ipython-input-8-b438e10836e5>, line 1)

In [17]:
tf.global_variables()

[<tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3_2/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3_3/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_3/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 